In [1]:
import lsst.afw.image as afwImage
import lsst.geom as geom
import glob
import sqlite3
from astropy.table import Table
import pandas as pd

In [2]:
!pwd

/pscratch/sd/s/shl159/projects/update_plots/dia_improvement/devel


In [3]:
# ! cp '/pscratch/sd/s/shl159/Cori/projects/fake_injection_v23/dia_improvement/devel/data/patch_0to6/diff/al_default_v23/detection/detection.sqlite' './default_detection.sqlite'

#### Postage Stamps vs. Spatial Degree of Freedom

In [4]:
x, y = 1000, 1000
magVar = 24
calexp_dir = '/pscratch/sd/s/shl159/Cori/projects/fake_injection_v23/dia_improvement/devel/data/patch_0to6/injection'
db = '/pscratch/sd/s/shl159/Cori/projects/fake_injection_v23/dia_improvement/devel/data/patch_0to6/diff/al_default_v23/detection/detection.sqlite'
conn = sqlite3.connect(db)
art = pd.read_sql_query("SELECT * FROM artifact WHERE host_mag = '20_21' and fake_mag = 20", conn)
art_astropy = Table.from_pandas(art)
for c, row in enumerate(art_astropy):
    if c == 1:
        break
    patch = str(row['patch'])
    det = int(row['detector'])
    filt = str(row['filter'])
    visit = int(row['visit'])
    calexp_path = glob.glob(f'{calexp_dir}/{patch}_20_21_{visit}_{det}_{filt}/fake_20/calexp/*/*/calexp*')[0]
    calexp_exp = afwImage.ExposureF(calexp_path)
    photoCalib = calexp_exp.getPhotoCalib()
    psf = calexp_exp.getPsf()
    xy = geom.Point2D(x, y)
    starIm = psf.computeImage(xy)
    calibFluxRadius = 12
    correctedFlux = psf.computeApertureFlux(calibFluxRadius, xy)
    starIm /= correctedFlux
    flux = photoCalib.magnitudeToInstFlux(magVar, xy)
    starIm *= flux
    star_shape = starIm.array.shape
    star_array = starIm.getArray()
    print((star_array < 1e-19).sum(), (star_array < 0).sum())
   

471 278


In [5]:
?psf

Type:           PsfexPsf
String form:    <lsst.meas.extensions.psfex.psfexPsf.PsfexPsf object at 0x7fb35ef1e530>
File:           /opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-0.7.0/Linux64/meas_extensions_psfex/g0683b0d77a+65487390d5/python/lsst/meas/extensions/psfex/psfexPsf.so
Docstring:      <no docstring>
Init docstring: __init__(self: lsst.meas.extensions.psfex.psfexPsf.PsfexPsf, psf: lsst.meas.extensions.psfex.psf.Psf, averagePosition: lsst.geom.Point2D = Point2D(0, 0)) -> None


In [6]:
print(star_array.shape)

(41, 41)


https://github.com/lsst/meas_extensions_psfex/blob/main/python/lsst/meas/extensions/psfex/psfex.py

In [8]:
calexp_info = calexp_exp.getInfo()

In [10]:
visit_info = calexp_info.getVisitInfo()

In [13]:
visit_info.exposureTime

30.0